# How to Clean Up Your Data
## Example: Where to look for Meteorites in the USA

So you have some fresh data, which you are sure contains useful insights. But now what?

While every dataset will need its own specific cleaning steps, we follow a general procedure, no matter the data we are cleaning. I will use as an example the Meteorite Landings dataset provided by the Meteorical Society, accessible at the NASA Open Data Portal (this webpage) [https://data.nasa.gov/Space-Science/Meteorite-Landings/ak9y-cwf9]. I will data clean using Python 3.9 in Microsoft Visual Studio Code. 

In any data cleaning exercise, we follow these steps:

### Step 1: Define your initial goal.

Carefully define what you want from the data. You may revise it, but at first, think about what you want to accomplish. For the Meteorite Landings data, we want to find where we can most readily find meteorites in the continental USA. Assuming that meteorite landings are spread evenly across the surface of the Earth, this would correspond to the area with the *least* density of meteorites which have been found.

### Step 2: Summarize your Dataset

This basic step involves doing very basic perusals of your data. How many data entries does the data contain? How many data fields? What does each data field contain? 

Pandas is our favorite library for answering these basic questions, because of its ease of use and many handy methods for data exploration. If you haven't already, create a virtual environment in your project directory, activate it, and then install pandas.

* python 3.9 -m venv .venv *(Create a Virtual Environment in your current directory)*
* .venv\Scripts\Activate.ps1 *(activate it in your PowerShell terminal)*
* pip install pandas *(install pandas!)*

Now open your data and inspect it using. I'll do this for Meteorite Landings as an example. This data is in CSV, handy to read using pandas -- pandas will read the CSV and convert into a tabular dataframe. I will inspect the summary information of this dataframe and then inspect a sample using two handy dataframe methods - sample(), and the atttribute .style.

In [21]:
import pandas as pd

meteorite_data = pd.read_csv('example-datasets/Meteorite_Landings.csv')

meteorite_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         45716 non-null  object 
 1   id           45716 non-null  int64  
 2   nametype     45716 non-null  object 
 3   recclass     45716 non-null  object 
 4   mass (g)     45585 non-null  float64
 5   fall         45716 non-null  object 
 6   year         45425 non-null  float64
 7   reclat       38401 non-null  float64
 8   reclong      38401 non-null  float64
 9   GeoLocation  38401 non-null  object 
 10  States       1659 non-null   float64
 11  Counties     1659 non-null   float64
dtypes: float64(6), int64(1), object(5)
memory usage: 4.2+ MB


The dataset is 4.2 MB in size, with 11 fields composed of a mix of strings and float values. 

#### Data Summary
Summarizing this work, we can see that this dataset contains a list of known meteorites which have struck the earth, along with their latitude, longitude, mass, and year of observation. 

### Step 4: Data Dictionary Review and Key Variable identification
**Goal** Understand all of the data in your dataset, and identify key variables that you will use in your analysis, and which you *might* use in your analysis. 

Now, review the fields, line by line, as they appear in the dataset. Also peruse the available documentation, to help understand each field. Once you have done that, give a line-by-line summary of what each data field describes and the data it contains. 

It's always important to review the "real" data when you do this, rather than just field names. So let's look at a random sample of 5 data points.

In [22]:
meteorite_data.sample(5).style

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,States,Counties
43611,Yamato 980172,36798,Valid,L6,480.630000,Found,1998.000000,nan,nan,nan,nan,nan
20291,Lewis Cliff 86383,13314,Valid,H5,10.500000,Found,1986.000000,-84.273730,161.693170,"(-84.27373, 161.69317)",nan,nan
11749,Fort Pierre,10167,Valid,"Iron, IIIAB",15900.000000,Found,1856.000000,44.350000,-100.383330,"(44.35, -100.38333)",21.000000,2732.000000
22696,MacAlpine Hills 87314,15257,Valid,L6,319.300000,Found,1987.000000,-84.216670,160.500000,"(-84.21667, 160.5)",nan,nan
10011,Elephant Moraine 87857,8407,Valid,L6,22.600000,Found,1987.000000,-76.282780,156.453910,"(-76.28278, 156.45391)",nan,nan


Here is a short summary of the fields, based on what we can observe, and from what the (very limited) documentation provides:

1) **name** is just a verbal description. This won't be much use in any analysis.
2) **id** appears to be a unique number for each field - pretty handy.
3) **nametype** it is not clear what this means, but if you look at the (webpage) [https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh] it says "Under NameType, 'valid' is for most meteorites and 'relict' are for objects that were once meteorites but are now highly altered by weathering on Earth." This field doesn't appear that useful to us.
4) **recclass** is ambiguous, and unfortunately the documentation doesn't describe what it means. (This is a problem data scientists have to live with - badly documented data.)
5) **mass** is the size of the meteorite.
6) **fall** is always either "Found" or "Fell" - telling us whether the meteorite was observed or actually found.
7) **year** is the year of being found.
8) **reclat**, **reclong**, and **GeoLocation** are all redundant fields expressing the location of the observation.
9) **States** and **Counties** are also ambiguous fields, composed of many numbers, and the documentation does not describe it any better.

Since we are interested simply in the spatial density of meteorite landings, our key variables are **reclat**, **reclong**, and **GeoLocation**. We also might use **year**, **id**, and **mass**, since these are interesting variables which could affect our analysis in ways we cannot yet anticipate. The variable **name** will also be useful as an identifier while we review the data.

Since we don't even understand what **nametype**, **reclass**, **States**, or **Counties** mean, we can't use them. 

It is also very hard to see how we will use **fall**. 


### Step 4: Explore your Data, Clean your Data, Explore your Data
Now begins a step with an end goal of preparing your data for the final analysis you which to do. This is generally an iterative process, where you first explore the data, first to find obvious issues needing correction, like:

* Formatting problems in your data needing correction.
* Standardization of fields to single, comparable units of a single comparable data type. 
* Blank or Null values, especially in your key variables, which must be dropped.

You should also apply any filters to focus on the data points you are interested in.

You can then do some more advanced exploration and cleaning. You should review the characteristics of your key variables, including statistics like median, maximum, minimum, but also other characteristics like clusters and intervariable correlations. This will help you identify outliers in your dataset which might need correction. 

There is no single recipe for how you find this information, but it will involve calculating statistics, reviewing histograms, making maps, and using other analysis techniques.

And once this exploration is complete, you apply various cleaning steps -- whether it be dropping Null values, correcting units, or otherwise. 

I will do this step by step for the example applied to the Meteorite Landings dataset.

#### Reviewing reclat, reclon, Geolocation rows with NaN
From visual inspection of data earlier, it is clear that many meteorites do not have location data. This makes these rows useless, and so I will drop NaN rows. But what is unclear is - are reclat, reclon, and Geolocation, completely redundant? Are there NaN rows in reclat, reclon, where data is available in Geolocation? We want to maximize the information. Let's review the NaN values. 


In [33]:
# Check and compare the number of NaN rows in each field. 
nan_rec_data=meteorite_data[meteorite_data['reclat'].isna()]
nan_geo_data=meteorite_data[meteorite_data['GeoLocation'].isna()]

print(f"The number of rows where reclat is NaN is {len(nan_rec_data)}, and where GeoLocation is NaN is {len(nan_geo_data)}.")

# Now create a dataframe whichi includes any instances where one value is NaN, but the other is not.
non_overlap_nan = meteorite_data[(meteorite_data['reclat'].isna()) & (meteorite_data['GeoLocation'].notna())]

non_overlap_nan = non_overlap_nan.append(meteorite_data[(meteorite_data['reclat'].notna()) & (meteorite_data['GeoLocation'].isna())])
print(f"The number of rows where only one location value is NaN is {len(non_overlap_nan)}")


The number of rows where reclat is NaN is 7315, and where GeoLocation is NaN is 7315.
The number of rows where only one location value is NaN is 0


C:\Users\14154\AppData\Local\Temp\ipykernel_22204\230847687.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_overlap_nan = non_overlap_nan.append(meteorite_data[(meteorite_data['reclat'].notna()) & (meteorite_data['GeoLocation'].isna())])


The location values of **reclat**, **reclon**, and **GeoLocation** are redundant - so we can drop all NaN values as useless.

In [34]:
print(f"Total tumber of meteorite strikes in the dataset: {len(meteorite_data)}.")
meteorite_data_c = meteorite_data.dropna(subset='reclat')
print(f"Total tumber of meteorite strikes containing location information: {len(meteorite_data_c)}.")

Total tumber of meteorite strikes in the dataset: 45716.
Total tumber of meteorite strikes containing location information: 38401.


Now, filter the strieks to only those in the USA> 